In [9]:
import pandas as pd

In [31]:
turnstile_files = ("MTA_Data/turnstile_200905.txt", "MTA_Data/turnstile_200912.txt", "MTA_Data/turnstile_200919.txt")
turnstile_data = (pd.read_csv(file) for file in turnstile_files)

In [32]:
turnstiles = pd.concat(turnstile_data)

In [34]:
turnstiles["DATE_TIME"] = pd.to_datetime(turnstiles['DATE'] + " " + turnstiles['TIME'])
one_day = pd.Timedelta(1, unit="D")
daily_turnstiles = turnstiles.loc[turnstiles['TIME'] == '00:00:00']
daily_turnstiles = daily_turnstiles[['C/A', 'UNIT', 'SCP', 'DATE_TIME', 'ENTRIES']]

In [58]:
grouped = daily_turnstiles.groupby(['C/A', 'UNIT', 'SCP', 'DATE_TIME'])
grouped.groups.iloc[0]

AttributeError: 'PrettyDict' object has no attribute 'iloc'